<a href="https://colab.research.google.com/github/ykitaguchi77/GO_AI_project/blob/main/Hertel_YOLOv5_MobileNetv3_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Hertel estimation YOLOv5-MobileNetv3 pipeline**

Train YOLOv5 using iFish augmentation

```
iFish: https://github.com/Gil-Mor/iFish.git

yolov5_iFish: https://github.com/ykitaguchi77/yolov5-iFish.git
※ FishAugment(distortion_range=(0, 0.3), p=0.5)

```

```
Olympia dataset
Dlibで目が2つ検出されるものを抜き出す
YOLOv5を用いて左右とバウンディングボックスを認識させる
抜き出した画像についてMobileNetV3で回帰（5-fold ensemble）を行う
スマホに実装
```

Output as CoreML

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import shutil
import copy
import pandas as pd
import csv
from random import randint
from time import sleep
import numpy as np
import sys
import torch
from tqdm import tqdm
import cv2
from google.colab.patches import cv2_imshow


import glob
import random
from PIL import Image
%matplotlib inline

# #サポートパッチのインポート
# from google.colab.patches import cv2_imshow
# import cv2

plt.ion()   # interactive mode
!nvidia-smi
print(torch.cuda.is_available())


Tue Nov  7 13:15:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#残り時間確認
!cat /proc/uptime | awk '{printf("残り時間 : %.2f", 12-$1/60/60)}'


残り時間 : 11.89

#Google colabをマウント

In [3]:
'''
・dlibを用いて目を切り抜く
・横幅を2倍、縦幅を上に1倍追加/下に0.5倍追加した両眼の画像が含まれるように切り取る（目の全幅、眉毛が含まれるように）
'''

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#親フォルダ
parent_dir = '/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv5'

#元画像フォルダ
dataset_dir = '/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset'

#元画像をコピー
orig_dir = f"{parent_dir}/dataset_orig"

#切りぬいた画像を保存するフォルダ
out_dir = f"{parent_dir}/dataset_uni"

#トレーニングされたYOLOv5で切り抜いた画像を保存するフォルダ
cropped_dir = f"{parent_dir}/dataset_yolo_cropped"

#CSVファイルのフォルダ
csv_hertel_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Hertel.csv"
csv_coordinate_path = f"{parent_dir}/coordinate_uni_for_YOLO5.csv"
csv_integrated_path = f"{parent_dir}/integrated_uni_for_YOLO5.csv"

In [ ]:
"""
###################################
# Reflesh folder (内容が削除されるので注意！！) #
###################################
"""

# parent_dirがあれば削除する
if os.path.exists(parent_dir):
    shutil.rmtree(parent_dir)

# 新しくparent_dirを作成する
os.makedirs(parent_dir)

# orig_dir, out_dirを新規に作成する
os.makedirs(orig_dir)
os.makedirs(out_dir)
os.makedirs(cropped_dir)

# orig_dirにdataset_dir直下のファイルをすべてコピーする
file_list = os.listdir(dataset_dir)
for filename in tqdm(file_list, desc="Copying files", unit="file"):
    src_path = os.path.join(dataset_dir, filename)
    dst_path = os.path.join(orig_dir, filename)
    shutil.copy(src_path, dst_path)

print("処理が完了しました。")


Copying files: 100%|██████████| 1016/1016 [00:12<00:00, 83.69file/s]

処理が完了しました。


#**HaarCascadeを用いて目を検出**

In [ ]:
# カスケードファイルのパス
eye_cascade_path = '/content/drive/My Drive/Deep_learning/haarcascade_eye.xml'

# カスケード分類器の特徴量取得
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

#**目が2つ以上検出されたものを抜き出す**

dlibで検出されたものから、上下左右に0.1倍ずつ拡大した範囲を抜き出している

In [ ]:
with open(csv_coordinate_path, 'w', newline='') as f:
        #fieldnames = ['Number', 'Folder', 'FileName']
        writer = csv.writer(f)
        writer.writerow(['id','img_path', 'side R/L', 'ex', 'ey', 'ew', 'eh'])  #header

        files = os.listdir(orig_dir)

        k=0
        for file in files:  #フォルダ数の分だけ
              file_path = f"{orig_dir}/{file}"
              id = os.path.splitext(os.path.basename(file_path))[0]

              img = cv2.imread(file_path)
              img2 = img.copy()

              # 画像グレースケール化
              grayscale_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

              #300pix以上のもので目に見えるものを抽出
              eye_list = eye_cascade.detectMultiScale(grayscale_img, minSize=(300, 300))

              # 眼検出判定
              if len(eye_list) >= 1:
                  print('目が' + str(len(eye_list)) +'個検出されました')
              else:
                  print("eye detection error")

              #画像の切り抜きと保存（2個以上検出の時に限る）
              if len(eye_list) >= 2:
                  for (ex, ey, ew, eh) in eye_list:
                      print(f"img_width: {img2.shape[1]}")
                      print("[ex,ey] = %d,%d [ew,eh] = %d,%d" %(ex, ey, ew, eh))
                      cv2.rectangle(img2, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
                      img_cropped = img[int(ey-0.1*eh): int(ey+1.1*eh), int(ex-0.1*ew): int(ex+1.1*ew)] #本来の切り抜きより幅の0.1倍ずつ水増しする
                      #img_cropped = img[int(ey): int(ey+eh), int(ex): int(ex+ew)]


                      if ex+eh*0.5 <= img2.shape[1]/2:
                          side = "R" #横幅の半分より左にあるのは右眼
                      else:
                          side = "L" #横幅の半分よりより右にあるのは左眼

                      print(f"side: {side}")
                      print("")

                      # Check if coordinates are within the image bounds
                      ey_start = max(int(ey - 0.1 * eh), 0)
                      ey_end = min(int(ey + 1.1 * eh), img.shape[0])
                      ex_start = max(int(ex - 0.1 * ew), 0)
                      ex_end = min(int(ex + 1.1 * ew), img.shape[1])

                      # Ensure we have a valid crop area
                      if ex_start < ex_end and ey_start < ey_end:
                          img_cropped = img[ey_start: ey_end, ex_start: ex_end]

                          # Now do the checks for the right/left side, write image and row
                          # ...

                          cv2.imwrite(f"{out_dir}/{id}_{side}.png", img_cropped)

                      #対応表の作成
                      writer.writerow([id, file_path, side, ex-round(ew*0.1), ey-round(eh*0.1), round(ew*1.2), round(eh*1.2)])

                      #cv2_imshow(img_cropped)
                  else:
                      pass


##**ここで、目以外が誤検出されているものを手動で抜き出して削除する**

```
coordinate_uni_for_YOLO5.csvから、削除して画像のパスが存在しなくなっている行を削除する

```

In [ ]:
import os
import pandas as pd

# DataFrameを読み込む
coordinates_df = pd.read_csv(csv_coordinate_path)

# 存在しない画像パスをチェックし、そのリストを保持する
nonexistent_paths = coordinates_df[~coordinates_df['img_path'].apply(os.path.exists)]

# 存在しない画像パスを表示
print("Nonexistent image paths:")
print(nonexistent_paths['img_path'])

# # 存在しない画像パスの行を削除
# coordinates_df = coordinates_df[coordinates_df['img_path'].apply(os.path.exists)]

# # 更新されたDataFrameを保存する
# coordinates_df.to_csv('coordinate_uni_for_YOLO5.csv', index=False)


Nonexistent image paths:
Series([], Name: img_path, dtype: object)


##**Dataframeの整理**

・ hertel_dfを参照して、coordinates_dfにヘルテル値を記入する

・idが"16_R, 16_L"という形式になるようにデータフレームを整理する

In [ ]:
import pandas as pd

# Assuming csv_coordinate_path and csv_hertel_path are defined paths to the CSV files
coordinates_df = pd.read_csv(csv_coordinate_path)
hertel_df = pd.read_csv(csv_hertel_path)

coordinates_df['Hertel'] = None

def get_hertel_value(row, hertel_df):
    id = row['id']
    side = row['side R/L']
    hertel_value = hertel_df.loc[hertel_df['number'] == id, side].values
    return hertel_value[0] if len(hertel_value) > 0 else None

# Use .copy() to ensure that you're working with a copy and not a view
coordinates_df['Hertel'] = coordinates_df.apply(lambda row: get_hertel_value(row, hertel_df), axis=1)

id_counts = coordinates_df.groupby('id')['side R/L'].value_counts().unstack()
valid_ids = id_counts[(id_counts['R'] == 1) & (id_counts['L'] == 1)].index

# Filter the DataFrame to only include these ids
# Use .copy() to avoid SettingWithCopyWarning when modifying this DataFrame later
coordinates_filtered_df = coordinates_df[coordinates_df['id'].isin(valid_ids)].copy()
coordinates_filtered_df.reset_index(drop=True, inplace=True)

coordinates_filtered_df.to_csv(csv_integrated_path, index=False)
coordinates_filtered_df.head()

,id,img_path,side R/L,ex,ey,ew,eh,Hertel
0,19,/content/drive/MyDrive/Deep_learning/Olympia_d...,R,107,557,835,835,15.0
1,19,/content/drive/MyDrive/Deep_learning/Olympia_d...,L,1513,547,850,850,16.0
2,20,/content/drive/MyDrive/Deep_learning/Olympia_d...,R,74,483,942,942,18.0
3,20,/content/drive/MyDrive/Deep_learning/Olympia_d...,L,1488,486,978,978,18.0
4,21,/content/drive/MyDrive/Deep_learning/Olympia_d...,R,147,555,868,868,19.0


In [ ]:
'''
画像パスの抽出（RLともに揃っているもの）
'''
coordinates_filtered_df = coordinates_filtered_df.drop_duplicates(subset='id', keep='first')
img_path_list = coordinates_filtered_df['img_path'].tolist()


'''
画像の分割 train:valid = 8:2
'''
random.seed(42)  # For reproducibility
random.shuffle(img_path_list)

split_index = int(0.8 * len(img_path_list))
train_img_paths = img_path_list[:split_index]
valid_img_paths = img_path_list[split_index:]

if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
os.makedirs(out_dir)

'''
フォルダの作成
'''
folders = ['train/images', 'train/labels', 'valid/images', 'valid/labels']
for folder in folders:
    os.makedirs(os.path.join(out_dir, folder))


'''
画像のコピー
'''
# Define paths for images directories
train_images_dir = os.path.join(out_dir, 'train/images')
valid_images_dir = os.path.join(out_dir, 'valid/images')

# Copy training images
for img_path in tqdm(train_img_paths, desc='Copying train images'):
    shutil.copy(img_path, train_images_dir)

# Copy validation images
for img_path in tqdm(valid_img_paths, desc='Copying valid images'):
    shutil.copy(img_path, valid_images_dir)

Copying valid images: 100%|██████████| 190/190 [00:02<00:00, 72.86it/s]


In [59]:
import os
import pandas as pd
from PIL import Image

def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height

def convert_to_yolo_format(ex, ey, ew, eh, img_width, img_height):
    cx = (ex + (ew / 2)) / img_width
    cy = (ey + (eh / 2)) / img_height
    w = ew / img_width
    h = eh / img_height
    return cx, cy, w, h

def create_label_files(image_dir, label_dir, df):
    image_files = os.listdir(image_dir)
    for image_file in tqdm(image_files, desc="Processing images"):
        image_path = os.path.join(image_dir, image_file)
        img_width, img_height = get_image_dimensions(image_path)

        base_name = os.path.splitext(image_file)[0]
        matched_rows = df[df['id'] == int(base_name)]

        if matched_rows.empty:
            raise ValueError(f"No matching id found for image {image_file}")

        label_file_path = os.path.join(label_dir, f"{base_name}.txt")
        with open(label_file_path, 'w') as label_file:
            for _, row in matched_rows.iterrows():
                ex = row['ex']
                ey = row['ey']
                ew = row['ew']
                eh = row['eh']
                cx, cy, w, h = convert_to_yolo_format(ex, ey, ew, eh, img_width, img_height)
                side = 0 if row['side R/L'] == 'R' else 1
                label_file.write(f" {side} {cx} {cy} {w} {h}\n")
                #label_file.write(f"{ex} {ey} {ew} {eh} {side}\n")

# CSVファイルの読み込み
csv_integrated_df = pd.read_csv(csv_integrated_path)

# trainとvalidのディレクトリパス
train_images_dir = os.path.join(out_dir, "train/images")
train_labels_dir = os.path.join(out_dir, "train/labels")
valid_images_dir = os.path.join(out_dir, "valid/images")
valid_labels_dir = os.path.join(out_dir, "valid/labels")

# trainディレクトリでラベルファイルを生成
create_label_files(train_images_dir, train_labels_dir, csv_integrated_df)

# validディレクトリでラベルファイルを生成
create_label_files(valid_images_dir, valid_labels_dir, csv_integrated_df)


Processing images: 100%|██████████| 190/190 [01:49<00:00,  1.73it/s]


In [ ]:
##############################
## バウンディングボックスのサンプル描画 ##
## (これは実行しなくて良い)            ##
##############################

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# バウンディングボックスを描画する関数
def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height

def draw_bounding_boxes(image_path, bboxes):
    with Image.open(image_path) as img:
        img_width, img_height = img.size
        fig, ax = plt.subplots(1)
        ax.imshow(img)
        for bbox in bboxes:
            cx, cy, bw, bh, class_id = bbox
            x = (cx - bw / 2) * img_width
            y = (cy - bh / 2) * img_height
            width = bw * img_width
            height = bh * img_height
            rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
        plt.show()

# ラベルファイルからバウンディングボックスのリストを取得する関数
def get_bboxes_from_label_file(label_path, img_width, img_height):
    bboxes = []
    with open(label_path, 'r') as file:
        for line in file:
            cx, cy, bw, bh, class_id = map(float, line.split())
            bboxes.append((cx, cy, bw, bh, class_id))
    return bboxes

# 画像パスとラベルファイルパス
image_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv5/dataset_uni/valid/images/10.JPG"
label_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv5/dataset_uni/valid/labels/10.txt"

# 画像のサイズを取得
img_width, img_height = get_image_dimensions(image_path)

# ラベルファイルからバウンディングボックスのリストを取得
bboxes = get_bboxes_from_label_file(label_path, img_width, img_height)


# バウンディングボックスを描画
print(f"img_width: {img_width}, img_height: {img_height}")
print(f"bboxes: {bboxes}")
draw_bounding_boxes(image_path, bboxes)


In [55]:
%cd $out_dir

/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv5/dataset_uni


In [56]:
%%writefile dataset.yaml
train: /content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv5/dataset_uni/train/images
val: /content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv5/dataset_uni/valid/images

nc: 2
names: ['right', 'left']

Overwriting dataset.yaml


#**Setup YOLOv5**

In [60]:
# # YOLOv5_iFish --> train_batchの精度が低いので一旦却下とした
# %cd $out_dir
# !git clone https://github.com/ykitaguchi77/yolov5-iFish.git #iFish augmentationを実装したバージョン
# %cd yolov5-iFish
# %pip install -qr requirements.txt

# import torch
# import utils
# display = utils.notebook_init()

YOLOv5 🚀 4d3e758 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (8 CPUs, 51.0 GB RAM, 28.2/166.8 GB disk)


In [5]:
# YOLOv5
%cd $out_dir
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

import torch
import utils
display = utils.notebook_init()

YOLOv5 🚀 v7.0-240-g84ec8b5 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (8 CPUs, 51.0 GB RAM, 28.2/166.8 GB disk)


#**Train YOLOv5**

In [ ]:
# Train
!python train.py --img 640 --batch 16 --epochs 300 --data $out_dir/dataset.yaml --weights yolov5n.pt

In [6]:
# 途中から
!python train.py --img 640 --batch 16 --epochs 300 --data $out_dir/data.yaml --resume $out_dir/yolov5/runs/train/exp/weights/last.pt

2023-11-07 13:15:58.172429: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-07 13:15:58.172484: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-07 13:15:58.172523: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv5/dataset_uni/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=640, rect=False, resume=/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv5/dataset_uni/yolov5/runs/train/exp/weights/last.pt, nosav

In [7]:
# best.pyをrenameしてgdriveに移動しておく
orig_pt = f"{out_dir}/yolov5/runs/train/exp/weights/best.pt"
dst_pt = f"{out_dir}/eyecrop-yolov5n-167epoch.pt"
shutil.copy(orig_pt, dst_pt)

'/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv5/dataset_uni/eyecrop-yolov5n-167epoch.pt'

#**Inference original dataset**

In [6]:
%cd $out_dir/yolov5

import torch
import utils
%pip install -qr requirements.txt
display = utils.notebook_init()

YOLOv5 🚀 4d3e758 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (8 CPUs, 51.0 GB RAM, 27.1/166.8 GB disk)


In [8]:
# YOLOv5
#weight = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training/yolov5-iFish/runs/train/exp2/weights/best.pt"
#weight = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training/yolov5n-iFish_120epoch.pt"
weight = f"{out_dir}/dataset_uni/eyecrop-yolov5n-iFish_169epoch.pt"
weight = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv5/dataset_uni/eyecrop-yolov5n-167epoch.pt"

# もともとのデータセット
orig_dir = orig_dir #元画像
cropped_dir = cropped_dir #YOLOv5で切り抜いた画像用

if os.path.exists(cropped_dir):
    shutil.rmtree(cropped_dir)
os.makedirs(cropped_dir)
os.makedirs(f"{cropped_dir}/cropped_images")


In [47]:
image_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv5/dataset_orig/1.JPG"

In [9]:
from models.common import DetectMultiBackend
#from utils.datasets import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr,
                           increment_path, non_max_suppression, print_args, strip_optimizer, xyxy2xywh)
#from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync
from utils.augmentations import letterbox #padding

from PIL import Image
import torch
from torchvision import models, transforms
import cv2
import numpy as np

#サポートパッチのインポート
from google.colab.patches import cv2_imshow


def inference(img, weight):
    device = 'cpu'
    device = select_device(device)
    model = DetectMultiBackend(weight, device=device, dnn=False)
    #stride, names, pt, jit, onnx, engine = model.stride, model.names, model.pt, model.jit, model.onnx, model.engine
    #imgsz = check_img_size([640], s=stride)  # check image size

    #class_names = {0:"cont", 1:"grav"}

    # transform = transforms.Compose([
    #             transforms.Resize(size=(480,640)),
    #             transforms.ToTensor(),
    #             # transforms.Normalize(
    #             #     mean=[0.5, 0.5, 0.5],
    #             #     std=[0.5, 0.5, 0.5]
    #             #    )
    #             ])

    img_cv2 = cv2.imread(img) #CV2で開く
    img_cv2 = letterbox(img_cv2, (640,640), stride=32, auto=False)[0] #resize, 上下padding (color 114)

    #cv2_imshow(img_cv2)

    img_cv2 = img_cv2.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    img_cv2 = np.ascontiguousarray(img_cv2)
    img_tensor = torch.from_numpy(img_cv2).float()

    #img_tensor = transform(img_np)
    img_tensor /= 255
    #print(img_tensor.shape)

    #print(img_tensor)
    img_tensor = torch.unsqueeze(img_tensor, 0)  # バッチ対応


    pred = model(img_tensor, visualize=False, augment=False)

    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None,  max_det=1000)

    print(f"pred: {pred}")

    return pred

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import glob


image_path = glob.glob(f"{orig_dir}/*")
start_index = 1
end_index = 5

class_names = {0: "right", 1: "left"}

for i in range(start_index, end_index + 1):
    img = image_path[i]
    pred = inference(img, weight)

    # output result
    x1, y1, x2, y2, prob, class_num = torch.round(pred[0][0])

    # probability
    prob = pred[0][0][4].item()

    # class
    class_name = class_names[pred[0][0][5].item()]

    print("診断は %s、確率は%.1f％です。" % (class_name, prob * 100))

    img_cv2 = cv2.imread(img)
    img_cv2 = cv2.resize(img_cv2, (640, int(img_cv2.shape[0] * 640 / img_cv2.shape[1])))  # 横幅を640pxにリサイズ

    # calculate coordinates of the bounding box (640*640にpaddingされている分の座標を足す)
    img_height, img_width, _ = img_cv2.shape[:3]
    print(f"img_height: {img_height}, img_width: {img_width}")
    padding_x = (img_height - min(img_width, img_height)) / 2
    padding_y = (img_width - min(img_width, img_height)) / 2
    x1 = x1 - padding_x
    y1 = y1 - padding_y
    x2 = x2 - padding_x
    y2 = y2 - padding_y
    print(f"x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}")

    # draw bounding box
    cv2.rectangle(img_cv2, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

    # show image
    cv2_imshow(img_cv2)

    # Crop and save the image
    cropped_img = img_cv2[int(y1):int(y2), int(x1):int(x2)]

    base_name = os.path.basename(img)
    cropped_img_path = os.path.join(cropped_dir, f"cropped_{base_name}")
    cv2.imwrite(cropped_img_path, cropped_img)


In [ ]:
"""""""""""""""""""""""""""""""""""""""
はみ出し防止、長辺を1辺としたletterbox化
"""""""""""""""""""""""""""""""""""""""

import os
import cv2
import numpy as np
import glob
from google.colab.patches import cv2_imshow

# もしcropped_dirが存在しない場合は作成する
if not os.path.exists(cropped_dir):
    os.makedirs(cropped_dir)

image_paths = glob.glob(f"{orig_dir}/*")
start_index = 1
end_index = 5

class_names = {0: "right", 1: "left"}

for i in range(start_index, end_index + 1):
    img_path = image_paths[i]
    pred = inference(img_path, weight)
    x1, y1, x2, y2, prob, class_num = torch.round(pred[0][0])
    prob = pred[0][0][4].item()
    class_name = class_names[pred[0][0][5].item()]

    print("診断は %s、確率は%.1f％です。" % (class_name, prob * 100))

    img_cv2 = cv2.imread(img_path)
    img_cv2 = cv2.resize(img_cv2, (640, int(img_cv2.shape[0] * 640 / img_cv2.shape[1])))

    img_height, img_width, _ = img_cv2.shape[:3]
    padding_x = (img_width - min(img_width, img_height)) / 2
    padding_y = (img_height - min(img_width, img_height)) / 2

    x1 = max(x1 - padding_x, 0)
    y1 = max(y1 - padding_y, 0)
    x2 = min(x2 - padding_x, img_width)
    y2 = min(y2 - padding_y, img_height)

    # クロップされた画像の実際のサイズ
    cropped_height = int(y2 - y1)
    cropped_width = int(x2 - x1)

    # 黒い背景画像を作成
    if cropped_width > cropped_height:
        size = cropped_width
    else:
        size = cropped_height
    padded_img = np.zeros((size, size, 3), dtype=np.uint8)

    # クロップした画像を中央に配置
    top = (size - cropped_height) // 2
    bottom = size - cropped_height - top
    left = (size - cropped_width) // 2
    right = size - cropped_width - left

    cropped_img = img_cv2[int(y1):int(y2), int(x1):int(x2)]
    padded_img[top:top + cropped_height, left:left + cropped_width] = cropped_img

    cv2.rectangle(img_cv2, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
    cv2_imshow(img_cv2)

    base_name = os.path.basename(img_path)
    cropped_img_path = os.path.join(cropped_dir, f"cropped_{base_name}")
    cv2.imwrite(cropped_img_path, padded_img)


In [40]:
"""""""""""""""""""""""""""""
640pxにリサイズしないバージョン
"""""""""""""""""""""""""""""

import os
import cv2
import numpy as np
import glob
from google.colab.patches import cv2_imshow

# もしcropped_dirが存在しない場合は作成する
if not os.path.exists(cropped_dir):
    os.makedirs(cropped_dir)

image_paths = glob.glob(f"{orig_dir}/*")
start_index = 1
end_index = 5

class_names = {0: "right", 1: "left"}


for i in range(start_index, end_index + 1):
    img_path = image_paths[i]
    pred = inference(img_path, weight)
    x1, y1, x2, y2, prob, class_num = pred[0][0]
    class_name = class_names[class_num]

    print(f"診断は {class_name}、確率は{prob * 100:.1f}%です。")

    img_cv2 = cv2.imread(img_path)

    # 画像から該当する領域を切り抜く
    cropped_img = img_cv2[int(y1):int(y2), int(x1):int(x2)]

    # 確認のために元画像に矩形を描画し、表示する
    cv2.rectangle(img_cv2, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
    cv2_imshow(img_cv2)

    base_name = os.path.basename(img_path)
    cropped_img_path = os.path.join(cropped_dir, f"cropped_{base_name}")
    cv2.imwrite(cropped_img_path, cropped_img)


YOLOv5 🚀 4d3e758 Python-3.10.12 torch-2.1.0+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 1761871 parameters, 0 gradients, 4.1 GFLOPs


pred: [tensor([[111.61232,  56.96371, 485.60168, 487.11520,   0.87449,   0.00000]])]
111.61231994628906 56.963714599609375 485.6016845703125 487.1152038574219


KeyError: ignored

#**Export coreML including non-max supression**

In [ ]:
# clone Yolov5 repo
import os
%cd /content
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt -r requirements.txt

In [ ]:
weight_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training/yolov5n-iFish_120epoch.pt"

In [ ]:
#!python export.py --weights /content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training/yolov5n-iFish_120epoch.pt --nms --include coreml
!python export.py --weights $weight_path --nms --include "coreml"


In [ ]:
# clone Yolov5 repo
import os
%cd /content
!git clone https://github.com/hietalajulius/yolov5.git
%cd yolov5
!pip install -r requirements.txt -r requirements-export.txt

In [ ]:
weight_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/yolo5n_100epoch.pt"

In [ ]:
!python export-nms.py --include coreml --weights $weight_path


#**Interference and crop Extended dataset**

In [ ]:
# Setup YOLOv5
%cd /content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

import torch
import utils
display = utils.notebook_init()

YOLOv5 🚀 v7.0-72-g064365d Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA A100-SXM4-40GB, 40536MiB)


Setup complete ✅ (12 CPUs, 83.5 GB RAM, 24.9/166.8 GB disk)


In [ ]:
# パスを指定する
weight = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/yolo5n_100epoch.pt"
input_folder = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/train/images"
output_folder = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_cropped_using_YOLO/train"

In [ ]:
from models.common import DetectMultiBackend
#from utils.datasets import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr,
                           increment_path, non_max_suppression, print_args, strip_optimizer, xyxy2xywh)
#from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync
from utils.augmentations import letterbox #padding

from PIL import Image
import torch
from torchvision import models, transforms
import cv2
import numpy as np

#サポートパッチのインポート
from google.colab.patches import cv2_imshow


def interference(img, weight):
    #stride, names, pt, jit, onnx, engine = model.stride, model.names, model.pt, model.jit, model.onnx, model.engine
    #imgsz = check_img_size([640], s=stride)  # check image size

    #class_names = {0:"cont", 1:"grav"}

    # transform = transforms.Compose([
    #             transforms.Resize(size=(480,640)),
    #             transforms.ToTensor(),
    #             # transforms.Normalize(
    #             #     mean=[0.5, 0.5, 0.5],
    #             #     std=[0.5, 0.5, 0.5]
    #             #    )
    #             ])

    img_cv2 = cv2.imread(img) #CV2で開く
    img_cv2 = letterbox(img_cv2, (640,640), stride=32, auto=False)[0] #resize, 上下padding (color 114)

    #cv2_imshow(img_cv2)

    img_cv2 = img_cv2.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    img_cv2 = np.ascontiguousarray(img_cv2)
    img_tensor = torch.from_numpy(img_cv2).float()

    #img_tensor = transform(img_np)
    img_tensor /= 255
    #print(img_tensor.shape)

    #print(img_tensor)
    img_tensor = torch.unsqueeze(img_tensor, 0)  # バッチ対応


    pred = model(img_tensor, visualize=False, augment=False)

    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None,  max_det=1000)

    print(f"pred: {pred}")

    return pred

In [ ]:
from tqdm import tqdm
import os

# image_path = glob.glob(f"{dataset_grav}/*")
# img = image_path[100]

class_names = {0:"cont", 1:"grav"}

device = 'cpu' # device = None or 'cpu' or 0 or '0' or '0,1,2,3'
device = select_device(device)
model = DetectMultiBackend(weight, device=device, dnn=False)

os.makedirs(output_folder, exist_ok=True)

for img in tqdm(glob.glob(f"{input_folder}/*")):

    pred = interference(img, weight)

    # output result
    x1, y1, x2, y2, prob, class_num = torch.round(pred[0][0])

    # probability
    prob = pred[0][0][4].item()

    # class
    class_name = class_names[pred[0][0][5].item()]

    print("診断は %s、確率は%.1f％です。" %(class_name, prob*100))

    img_cv2 = cv2.imread(img)

    # calculate coordinates of the bounding box (640*640にpaddingされている分の座標を足す)
    img_height, img_width, _ = img_cv2.shape[:3]
    print(f"img_height: {img_height}, img_width: {img_width}")
    padding_x = (img_height - min(img_width, img_height))/2
    padding_y = (img_width - min(img_width, img_height))/2
    x1 = x1 - padding_x
    y1 = y1 - padding_y
    x2 = x2 - padding_x
    y2 = y2 - padding_y
    print(f"x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}")

    # draw bounding box
    #cv2.rectangle(img_cv2, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

    # show image
    #cv2_imshow(img_cv2)

    # バウンディングボックスで画像を切り抜く」

    if x1 < 0: #負の場合のエラー回避
        x1 = 0

    cropped_image = img_cv2[int(y1):int(y2), int(x1):int(x2)]

    # 切り抜いた画像を保存する
    save_path = f"{output_folder}/{os.path.basename(img)}"
    print(save_path)
    #cv2_imshow(cropped_image)
    cv2.imwrite(save_path, cropped_image)

In [ ]:
#rewrite csv file (bootcamp用csvのimage_pathを改変)
import pandas as pd

csv_1_orig = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/train_list.csv"
csv_2_orig = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/valid_list.csv"
csv_1 = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_cropped_using_YOLO/train_list.csv"
csv_2 = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_cropped_using_YOLO/valid_list.csv"

def rewrite_csv(df):
    path_list = []
    for path in df["image_path"]:
        path = path.replace("periocular_for_YOLO_training", "periocular_cropped_using_YOLO")
        path = path.replace("images/", "")
        path_list.append(path)
    df["image_path"] = path_list
    return(df)

df = pd.read_csv(csv_1_orig)
df = rewrite_csv(df)
print(df)
df.to_csv(csv_1, index=False)

df = pd.read_csv(csv_2_orig)
df = rewrite_csv(df)
df.to_csv(csv_2,  index=False)

#**YOLOv5-EigenCAM**

https://jacobgil.github.io/pytorch-gradcam-book/EigenCAM%20for%20YOLO5.html

In [ ]:
%cd /content
!git clone https://github.com/jacobgil/pytorch-grad-cam.git

/content
Cloning into 'pytorch-grad-cam'...
remote: Enumerating objects: 1122, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 1122 (delta 8), reused 9 (delta 3), pack-reused 1097
Receiving objects: 100% (1122/1122), 110.21 MiB | 15.74 MiB/s, done.
Resolving deltas: 100% (617/617), done.


In [ ]:
%cd /content/pytorch-grad-cam

/content/pytorch-grad-cam


In [ ]:
!pip install pytorch-gradcam

In [ ]:
!pip install ttach

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import torch
import cv2
import numpy as np
import requests
import torchvision.transforms as transforms
from pytorch_grad_cam import EigenCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, scale_cam_image
from PIL import Image

COLORS = np.random.uniform(0, 255, size=(80, 3))

def parse_detections(results):
    detections = results.pandas().xyxy[0]
    detections = detections.to_dict()
    boxes, colors, names = [], [], []

    for i in range(len(detections["xmin"])):
        confidence = detections["confidence"][i]
        if confidence < 0.2:
            continue
        xmin = int(detections["xmin"][i])
        ymin = int(detections["ymin"][i])
        xmax = int(detections["xmax"][i])
        ymax = int(detections["ymax"][i])
        name = detections["name"][i]
        category = int(detections["class"][i])
        color = COLORS[category]

        boxes.append((xmin, ymin, xmax, ymax))
        colors.append(color)
        names.append(name)
    return boxes, colors, names


def draw_detections(boxes, colors, names, img):
    for box, color, name in zip(boxes, colors, names):
        xmin, ymin, xmax, ymax = box
        cv2.rectangle(
            img,
            (xmin, ymin),
            (xmax, ymax),
            color,
            2)

        cv2.putText(img, name, (xmin, ymin - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2,
                    lineType=cv2.LINE_AA)
    return img


image_url = "https://www.thesprucepets.com/thmb/3ABKoAPm0Hu4PcWsDH1giawq7ck=/750x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/chinese-dog-breeds-4797219-hero-2a1e9c5ed2c54d00aef75b05c5db399c.jpg"
img = np.array(Image.open(requests.get(image_url, stream=True).raw))
img = cv2.resize(img, (640, 640))
rgb_img = img.copy()
img = np.float32(img) / 255
transform = transforms.ToTensor()
tensor = transform(img).unsqueeze(0)

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.eval()
model.cpu()
target_layers = [model.model.model.model[-2]]

results = model([rgb_img])
boxes, colors, names = parse_detections(results)
detections = draw_detections(boxes, colors, names, rgb_img.copy())
Image.fromarray(detections)

# **To do Next**

・外部のデータセット（Treated）を洗い出し

・内部のデータセットをさらに水増し

・内部および外部データセットより、test用各100枚（grav50枚、cont50枚）を抜き出しておき、合体する

・既存のYOLOv5を用いてbounding boxを抜き出し、新たにトレーニングする